In [5]:
import cv2
import pyttsx3
import numpy as np

# Load the pre-trained Haar Cascade classifiers for pedestrian and full-body detection
pedestrian_cascade = cv2.CascadeClassifier('fullbody.xml')
fullbody_cascade = cv2.CascadeClassifier('upperbody.xml')

# Load the Haar Cascade classifier for hand detection (replace 'hand.xml' with your file)
hand_cascade = cv2.CascadeClassifier('hand.xml')

# Load video file or capture from webcam
video_source = "people.mp4"  # Set the video source path or use 0 for webcam
cap = cv2.VideoCapture(video_source)

# Initialize variables
people_count = 0
group_count = 0
group_threshold = 10  # Threshold for considering a group
message = ""
crowd_message = "The area is crowded."  # Message to be converted to speech
normal_message = "The area is in normal state."  # Message to be converted to speech

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Initialize message counters
crowd_message_count = 0
normal_message_count = 0

# Parameters for hand posture detection
lower_degrees = 90  # Lower threshold for hand posture (degrees)
upper_degrees = 150  # Upper threshold for hand posture (degrees)

def speak_message(message):
    engine.say(message)
    engine.runAndWait()

# Placeholder for crime detection
def detect_crime(frame):
    # Replace this with your crime detection logic
    # For example, you can use a pre-trained model or custom code
    return False

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale for pedestrian and full-body detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect pedestrians in the frame
    pedestrians = pedestrian_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Detect full-body in the frame
    fullbodies = fullbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Detect hands in the frame
    hands = hand_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Combine the results from all detectors
    all_people = list(pedestrians) + list(fullbodies)
    all_hands = list(hands)

    # Initialize variables for each frame
    frame_group_count = 0

    # Draw rectangles around detected people
    for (x, y, w, h) in all_people:
        if w * h > 1000:  # Filter out small detections
            if frame_group_count == 0:
                group_count += 1
            frame_group_count += 1
            color = (0, 0, 255) if frame_group_count > group_threshold else (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Draw rectangles around detected hands
    for (x, y, w, h) in all_hands:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Hand posture detection
    hand_posture_detected = False
    for (x, y, w, h) in all_hands:
        # Assuming the hand is detected at (x, y) and has width w and height h
        # Calculate the angle between the wrist and the tip of the middle finger
        hand_center_x = x + w // 2
        hand_tip_x = x + w
        hand_center_y = y + h // 2
        hand_tip_y = y
        angle_degrees = np.degrees(np.arctan2(hand_center_y - hand_tip_y, hand_tip_x - hand_center_x))

        if lower_degrees < angle_degrees < upper_degrees:
            hand_posture_detected = True
            break

    if hand_posture_detected:
        message = "Suspicious Activity Detected (Raised Hand)"
        if 'speech' not in message.lower():
            speak_message(message)

    # Placeholder for crime detection
    if detect_crime(frame):
        message = "Suspicious Activity Detected"
        if 'speech' not in message.lower():
            speak_message(message)

    # Update the people count and message
    people_count = len(all_people)
    
    if people_count > 20 and crowd_message_count < 1:
        message = "Crowd Formed"
        if 'speech' not in message.lower():
            speak_message(crowd_message)
            crowd_message_count += 1
        normal_message_count = 0
    elif people_count <= 20 and normal_message_count < 1:
        message = "Normal State"
        if 'speech' not in message.lower():
            speak_message(normal_message)
            normal_message_count += 1
        crowd_message_count = 0
    else:
        message = ""

    # Display the count and message on the frame
    cv2.putText(frame, f"People Count: {people_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, message, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the frame with detected people, hands, and hand posture
    cv2.imshow("People Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
